In [1]:
import numpy as np
import pandas as pd

In [4]:
# test = pd.read_csv('Source/test.txt', delimiter='\t')
train = pd.read_csv('Source/train', delimiter='\t')

In [5]:
train_0 = train


In [6]:
# y_train = train.DEF
# x_train = train[['CLIENT_ID', 'RETRO_DT', 'tokens', 'urls_hashed']]

In [7]:
train.head()

,CLIENT_ID,RETRO_DT,tokens,DEF,urls_hashed
0,5909886,20200911,форм 1 мачеха 1 работы 1 пасынка 1 горячую 1 м...,0,7bbcc146c0ba4df814701ff7297b2205 1 ee0d49cbc99...
1,1385448,20210519,platiza 1 мобильные 2 sergej 1 code 3 отказов ...,1,1889e0cbd3e343e9521a07aa312257ac 2 8253d8d3a67...
2,1866195,20210913,sims 3 кредит 1 языке 1 разлива 1 огромное 1 о...,0,6e617b4bfdbef85d8b09ec08ac72f0f2 1 94309f5f116...
3,1587081,20210702,code 1 историй 1 scf 1 шаг 1 деньги 3 серый 1 ...,0,1e833434273e04ba76cfcfb4b48ad21b 3 aee71c8d18d...
4,1400908,20210522,отказов 2 tovara 1 сергея 1 купальный 1 чёрный...,0,83e85bee0d81c0e9e5047e202cddf534 3 6e617b4bfdb...


In [11]:
test.head()

,CLIENT_ID,RETRO_DT,tokens,urls_hashed
0,5467000,20200419,vezet 1 hats 2 сухого 1 деньги 1 режим 4 недел...,4104c0437f707b9f443bf845d92d3633 1 83e85bee0d8...
1,6093848,20201107,статью 1 month 4 udobreniya 1 podkormka 1 акту...,db0853001fff2bbe88cb5fa709d8077e 1 6e617b4bfdb...
2,5269916,20200305,zdorovye 2 мобильные 1 снять 2 tovara 1 платье...,1889e0cbd3e343e9521a07aa312257ac 2 aee71c8d18d...
3,816015821,20220115,шины 1 воздушный 1 вкусных 1 эскудо 1 сухого 1...,aee71c8d18d6b91b36bf599208a91777 7 6754d470b22...
4,223498561,20211204,купить 2 blyudo 2 юрист 2 sekretnye 2 funktsii...,1246f8d1136a114c448c1b2565e16698 2 0f7f6a4507e...


In [12]:
print(len(list(train['CLIENT_ID'].unique())+ list(test['CLIENT_ID'].unique())))
print(len(set(list(train['CLIENT_ID'].unique())+ list(test['CLIENT_ID'].unique()))))

458581
458581


In [8]:
train.index = train.CLIENT_ID
train = train.drop(['CLIENT_ID'], axis = 1)
train.head()

,RETRO_DT,tokens,DEF,urls_hashed
CLIENT_ID,,,,
5909886,20200911,форм 1 мачеха 1 работы 1 пасынка 1 горячую 1 м...,0,7bbcc146c0ba4df814701ff7297b2205 1 ee0d49cbc99...
1385448,20210519,platiza 1 мобильные 2 sergej 1 code 3 отказов ...,1,1889e0cbd3e343e9521a07aa312257ac 2 8253d8d3a67...
1866195,20210913,sims 3 кредит 1 языке 1 разлива 1 огромное 1 о...,0,6e617b4bfdbef85d8b09ec08ac72f0f2 1 94309f5f116...
1587081,20210702,code 1 историй 1 scf 1 шаг 1 деньги 3 серый 1 ...,0,1e833434273e04ba76cfcfb4b48ad21b 3 aee71c8d18d...
1400908,20210522,отказов 2 tovara 1 сергея 1 купальный 1 чёрный...,0,83e85bee0d81c0e9e5047e202cddf534 3 6e617b4bfdb...


In [14]:
test.index = test.CLIENT_ID
test = test.drop(['CLIENT_ID'], axis = 1)
len(test)

154804

In [9]:
train0 = train[:1000]

In [10]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import time
time.perf_counter() 

17959.269434957

In [11]:
def all_tok(df):
    words = []
    nums = []  
    for i in df['tokens']:
        try: 
            words += i.split()[::2]
            nums += i.split()[1::2]
        except: pass
    for i in range(len(words)):
        words[i] = morph.parse(words[i])[0].normal_form
        nums[i] = int(nums[i])
        
    df = pd.DataFrame({"words": np.array(words), 'nums': np.array(nums)})
    return df
s = time.perf_counter() 
df = all_tok(train0)
print(time.perf_counter()-s)
df

65.03526538600272


,words,nums
0,форма,1
1,мачеха,1
2,работа,1
3,пасынок,1
4,горячий,1
...,...,...
642292,отправление,1
642293,sdat,2
642294,otets,1
642295,god,2


In [16]:
dft = df.groupby(['words'])['nums'].agg('sum').reset_index()
w_all = np.array(dft.sort_values(['nums'], ascending=0)['words'])

In [23]:
dft.sort_values(['nums'], ascending=0)[500:502]

,words,nums
18731,short,547
11870,native,547


In [56]:
train1 = train[train.DEF==1]
train1 = train1[:5000]

In [57]:
df = all_tok(train1)
dft = df.groupby(['words'])['nums'].agg('sum').reset_index()
w_1 = np.array(dft.sort_values(['nums'], ascending=0)['words'])

In [58]:
train2 = train[train.DEF==0]
train2 = train2[:5000]
df = all_tok(train2)
dft = df.groupby(['words'])['nums'].agg('sum').reset_index()
w_2 = np.array(dft.sort_values(['nums'], ascending=0)['words'])

In [59]:
w_2[:12]

array(['заём', 'карта', 'смотреть', 'новость', 'купить', 'магазин',
       'личный', 'цена', 'кабинет', 'новый', 'россия', 'почта'],
      dtype=object)

In [60]:
w_1[:12]

array(['заём', 'карта', 'личный', 'смотреть', 'кабинет', 'embed', 'новый',
       'бесплатно', 'россия', 'купить', 'новость', 'заявка'], dtype=object)

In [29]:
w_all[:12]

array(['заём', 'карта', 'личный', 'embed', 'кабинет', 'смотреть',
       'купить', 'россия', 'магазин', 'новый', 'цена', 'новость'],
      dtype=object)

In [55]:
set(w_1[:30])-set(w_2[:30])

{'client', 'билет', 'деньга', 'оформить'}

In [ ]:
def all_tok(df):
    words = []
    nums = []
    for i in df['tokens']:
        try: 
            words += i.split()[::2]
            nums += i.split()[1::2]
        except: pass
    for i in range(len(words)):
        words[i] = morph.parse(words[i])[0].normal_form
        nums[i] = int(nums[i])
    df = pd.DataFrame({"words": np.array(words), 'nums': np.array(nums)})
    return df

In [84]:
def token(s):
    lst = s.split()
    words = lst[::2]
    nums = lst[1::2]
    dic = {}
    for i in range(len(words)): 
        try: dic[morph.parse(words[i])[0].normal_form] += int(nums[i])
        except: dic[morph.parse(words[i])[0].normal_form] = int(nums[i])
    # print(words)
    print(dic)
    
token(train0['tokens'][5909886])

{'форма': 2, 'мачеха': 4, 'работа': 2, 'пасынок': 3, 'горячий': 1, 'массаж': 1, 'kakaya': 1, 'tovari': 1, 'дмб': 1, 'milf': 2, 'istoriya': 1, 'парень': 4, 'хуй': 1, 'voyska': 1, 'cvet': 1, 'род': 1, 'получать': 1, 'слушать': 1, 'molodym': 1, 'porno': 3, 'телепрограмма': 1, 'видеть': 1, 'полоса': 1, 'красивый': 1, 'сделать': 1, 'горячее': 1, 'мама': 1, 'колледж': 1, 'замужний': 1, 'varit': 1, 'kendra': 1, 'brigada': 1, 'луна': 1, 'цвет': 1, 'lust': 1, 'зрелый': 12, 'зависимость': 1, 'мужик': 1, 'тельняшка': 2, 'sportivnye': 1, 'баба': 2, 'завтра': 1, 'молодой': 8, 'американский': 1, 'качество': 2, 'дом': 1, 'отличаться': 1, 'заставить': 1, 'мать': 1, 'видео': 8, 'история': 1, 'clothes': 1, 'активация': 1, 'zavisimosti': 1, 'вместо': 1, 'porn': 2, 'неделя': 1, 'летний': 1, 'бесплатно': 1, 'грудастый': 1, 'предпочитать': 1, 'смотреть': 9, 'пойти': 1, 'video': 8, 'viewmsg': 1, 'отдельный': 1, 'наказание': 1, 'беспла': 1, 'порно': 8, 'mama': 1, 'негр': 1, 'полный': 1, 'tube': 1, 'массажист'

{'форма': '1', 'мачеха': '2', 'работа': '1', 'пасынок': '2', 'горячий': '1', 'массаж': '1', 'kakaya': '1', 'tovari': '1', 'дмб': '1', 'milf': '2', 'istoriya': '1', 'парень': '3', 'хуй': '1', 'voyska': '1', 'cvet': '1', 'род': '1', 'получать': '1', 'слушать': '1', 'molodym': '1', 'porno': '3', 'телепрограмма': '1', 'видеть': '1', 'полоса': '1', 'красивый': '1', 'сделать': '1', 'горячее': '1', 'мама': '1', 'колледж': '1', 'замужний': '1', 'varit': '1', 'kendra': '1', 'brigada': '1', 'луна': '1', 'цвет': '1', 'lust': '1', 'зрелый': '7', 'зависимость': '1', 'мужик': '1', 'тельняшка': '1', 'sportivnye': '1', 'баба': '1', 'завтра': '1', 'молодой': '1', 'американский': '1', 'качество': '2', 'дом': '1', 'отличаться': '1', 'заставить': '1', 'мать': '1', 'видео': '8', 'история': '1', 'clothes': '1', 'активация': '1', 'zavisimosti': '1', 'вместо': '1', 'porn': '2', 'неделя': '1', 'летний': '1', 'бесплатно': '1', 'грудастый': '1', 'предпочитать': '1', 'смотреть': '1', 'пойти': '1', 'video': '8', '

'yyy'

In [ ]:
y_train.to_csv('Source/y_train.csv', index = False)
x_train.to_csv('Source/x_train.csv', index = False)
test.to_csv('test.csv')

In [ ]:
def reverse_translit_2(word):
    i = 0
    lenght = len(word)
    new_word = ""
    while(i < lenght):
        symbol = word[i]
        if symbol == "a":
            new_word += "а"

        elif symbol == "b":
            new_word += "б"

        elif symbol == "v":
            new_word += "в"

        elif symbol == "g":
            new_word += "г"
        
        elif symbol == "h":
            new_word += "х"

        elif symbol == "d":
            new_word += "д"

        elif symbol == "e":
            if (i + 1) < lenght and word[i + 1] == "h":
                new_word += "э"
                i += 1
            else:
                new_word += "е"

        elif symbol == "y":
            if (i + 1) < lenght and word[i + 1] == "a":
                new_word += "я"
                i += 1
            elif (i + 1) < lenght and word[i + 1] == "u":
                new_word += "ю"
                i += 1
            elif (i + 1) < lenght and word[i + 1] == "h":
                new_word += "ь"
                i += 1
            elif (i + 1) < lenght and word[i + 1] == "o":
                new_word += "ё"
                i += 1
            elif (i + 1) < lenght and word[i + 1] == "e":
                new_word += "е"
                i += 1
            else:
                new_word += "й"
            if (i + 2) < lenght and word[i + 1] == word[i + 2] == "h":
                new_word += "ъ"
                i += 2

        elif symbol == "z":
            if (i + 1) < lenght and word[i + 1] == "h":
                new_word += "ж"
                i += 1
            else:
                new_word += "з"

        elif symbol == "i":
            if (i + 1) < lenght and word[i + 1] == "h":
                new_word += "ы"
                i += 1
            else:
                new_word += "и"

        elif symbol == "k":
            new_word += "к"

        elif symbol == "l":
            new_word += "л"

        elif symbol == "m":
            new_word += "м"

        elif symbol == "n":
            new_word += "н"

        elif symbol == "o":
            new_word += "о"

        elif symbol == "p":
            new_word += "п"

        elif symbol == "r":
            new_word += "р"

        elif symbol == "s":
            if (i + 1) < lenght and word[i + 1] == "h":
                if (i + 2) < lenght and word[i + 2] == "h":
                    new_word += "щ"
                    i += 2
                else:
                    new_word += "ш"
                    i += 1
            else:
                    new_word += "с"

        elif symbol == "t":
            new_word += "т"

        elif symbol == "u":
            new_word += "у"

        elif symbol == "f":
            new_word += "ф"

        elif symbol == "c":
            if (i + 1) < lenght and word[i + 1] == "h":
                new_word += "ч"
                i += 1
            else:
                new_word += "ц"

        i += 1
    return(new_word)